# Simple Deep Learning Neural Net using Keras
The goal of this notebook is to define and train a very simple NN using Keras, make some health related prediction using a given dataset, and play a little bit with the parameters and plots. Contrary to the other repo, we'll split the dataset into train and test sets.
* Sources : 
    * https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
    * https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

### Installing libraries

In [2]:
import sys
!{sys.executable} -m pip install keras numpy tensorflow

You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### Importing libraries

In [25]:
from keras.models import Model
from keras.layers import Dense, Input
import numpy

### Loading data
This dataset has :
* 768 rows or 'examples'
* 9 columns

In [26]:
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
rows, columns = dataset.shape
print('Rows :',rows)
print('Columns :', columns)

Rows : 768
Columns : 9


### Dataset columns
We'll split it into train and test sets, with the respective proportions 90% : 10%.
#### Input X :
- Number of times pregnant
- Plasma glucose concentration a 2 hours in an oral glucose tolerance test
- Diastolic blood pressure (mm Hg)
- Triceps skin fold thickness (mm)
- 2-Hour serum insulin (mu U/ml)
- Body mass index (weight in kg/(height in m)^2)
- Diabetes pedigree function
- Age (years)

#### Output Y :
- Class variable (0 or 1) 

In [27]:
train_set_size = int(rows*.9)

# split into input (X) and output (Y) variables for train and test sets.
# SOMETIME, TRY USING 'validation_split' IN FIT FUNCTION INSTEAD.
X_train = dataset[:train_set_size,0:8]
X_test = dataset[train_set_size+1:,0:8]
Y_train = dataset[:train_set_size,8]
Y_test = dataset[train_set_size+1:,8]

### Creating the model
* We have 8 inputs from X
* 1 output in Y
* We'll use 3 fully connected layers, the first having 12 neurons for 8 inputs, then 8 neurons, and finally 1 for the binary output prediction.
* We'll leave the default weight initialization.

In [38]:
def MyModel(input_shape):
    X_input = Input(input_shape)
    X = Dense(12, activation='relu', name='Layer1')(X_input)
    X = Dense(8, activation='relu', name='Layer2')(X)
    X = Dense(1, activation='sigmoid', name='Layer3')(X)
    return Model(inputs=X_input, outputs=X, name='my model')

input_shape = (X_train.shape[1],)
model = MyModel(input_shape)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Training the Neural Net
* A.k.a fitting the model.
* `epochs` is the number of times the training data is passed through the neural net. 
* `batch_size` is the number of training example let throught the net at each forward/backward propagation step. Weights are updated after each step. Convergence should be faster.
* Verbosity is turned off.

In [39]:
model.fit(X_train, Y_train, epochs=150, batch_size=10, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/150
691/691 [==============================] - 1s 1ms/step - loss: 6.5773 - acc: 0.4139
Epoch 2/150
691/691 [==============================] - 0s 201us/step - loss: 4.0360 - acc: 0.5340
Epoch 3/150
691/691 [==============================] - 0s 207us/step - loss: 0.7520 - acc: 0.6512
Epoch 4/150
691/691 [==============================] - 0s 198us/step - loss: 0.6760 - acc: 0.6628
Epoch 5/150
691/691 [==============================] - 0s 196us/step - loss: 0.6486 - acc: 0.6614
Epoch 6/150
691/691 [==============================] - 0s 204us/step - loss: 0.6428 - acc: 0.6657
Epoch 7/150
691/691 [==============================] - 0s 197us/step - loss: 0.6370 - acc: 0.6787
Epoch 8/150
691/691 [==============================] - 0s 197us/step - loss: 0.6341 - acc: 0.6744
Epoch 9/150
691/691 [==============================] - 0s 222us/step - loss: 0.6317 - acc: 0.6715
Epoch 10/150
691/691 [==============================] - 0s 209us/step - 

691/691 [==============================] - 0s 199us/step - loss: 0.5458 - acc: 0.7265
Epoch 81/150
691/691 [==============================] - 0s 188us/step - loss: 0.5495 - acc: 0.7149
Epoch 82/150
691/691 [==============================] - 0s 194us/step - loss: 0.5439 - acc: 0.7265
Epoch 83/150
691/691 [==============================] - 0s 196us/step - loss: 0.5493 - acc: 0.7135
Epoch 84/150
691/691 [==============================] - 0s 190us/step - loss: 0.5456 - acc: 0.7221
Epoch 85/150
691/691 [==============================] - 0s 195us/step - loss: 0.5468 - acc: 0.7221
Epoch 86/150
691/691 [==============================] - 0s 192us/step - loss: 0.5444 - acc: 0.7250
Epoch 87/150
691/691 [==============================] - 0s 193us/step - loss: 0.5443 - acc: 0.7221
Epoch 88/150
691/691 [==============================] - 0s 192us/step - loss: 0.5457 - acc: 0.7265
Epoch 89/150
691/691 [==============================] - 0s 190us/step - loss: 0.5418 - acc: 0.7279
Epoch 90/150
691/691 [=

### Evaluating the Neural Net
This is performed on the test set, which wasn't used for training.

In [41]:
# evaluate the model
scores = model.evaluate(X_test, Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

76/76 [==============================] - 0s 86us/step

acc: 65.79%


### Making predictions
Predictions are rounded towards 0 or 1 to get a clean result, because as the output comes from the sigmoid function, it will be some value anywhere between 0 and 1.

In [42]:
# calculate predictions
predictions = model.predict(X_test)
# round predictions
rounded = [round(x[0]) for x in predictions]
# show the 20 first predictions
print(rounded[:20], '...')

[0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0] ...
